# Applying LSTM Models on Raw Data

In [ ]:
# Importing Libraries

import pandas as pd
import numpy as np

# Import Keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2

In [ ]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# function to print the confusion matrix

def confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

    
    result = confusion_matrix(Y_true, Y_pred)

    plt.figure(figsize=(10, 8))
    sns.heatmap(result, 
                xticklabels= list(ACTIVITIES.values()), 
                yticklabels=list(ACTIVITIES.values()), 
                annot=True, fmt="d");
    plt.title("Confusion matrix")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show() 

### Loading Data

In [ ]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

In [ ]:
# define a function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/MyDrive/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [ ]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/MyDrive/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [ ]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [ ]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [ ]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [ ]:
#function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [ ]:
# Loading the train and test data
X_train, X_test, y_train, y_test = load_data()

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


 1. Defining the Architecture of 1-Layer of LSTM

In [ ]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 32)                5376      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test, y_test),epochs=epochs)

Epoch 1/30
460/460 [==============================] - 8s 11ms/step - loss: 1.0781 - accuracy: 0.5423 - val_loss: 0.9016 - val_accuracy: 0.6213
Epoch 2/30
460/460 [==============================] - 4s 10ms/step - loss: 0.7431 - accuracy: 0.6632 - val_loss: 0.6547 - val_accuracy: 0.7255
Epoch 3/30
460/460 [==============================] - 5s 12ms/step - loss: 0.5874 - accuracy: 0.7629 - val_loss: 0.5316 - val_accuracy: 0.7906
Epoch 4/30
460/460 [==============================] - 4s 8ms/step - loss: 0.4666 - accuracy: 0.8320 - val_loss: 1.4815 - val_accuracy: 0.6155
Epoch 5/30
460/460 [==============================] - 4s 8ms/step - loss: 0.3551 - accuracy: 0.8863 - val_loss: 0.4364 - val_accuracy: 0.8483
Epoch 6/30
460/460 [==============================] - 6s 12ms/step - loss: 0.2936 - accuracy: 0.9067 - val_loss: 0.3727 - val_accuracy: 0.8765
Epoch 7/30
460/460 [==============================] - 4s 8ms/step - loss: 0.2671 - accuracy: 0.9174 - val_loss: 0.4289 - val_accuracy: 0.8639
Ep

In [ ]:
# Confusion Matrix
confusion_matrix(y_test, model.predict(X_test))

93/93 [==============================] - 1s 4ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,511,0,26,0,0,0
SITTING,1,409,76,4,0,1
STANDING,0,95,433,4,0,0
WALKING,0,0,0,480,15,1
WALKING_DOWNSTAIRS,0,0,0,0,419,1
WALKING_UPSTAIRS,0,1,0,12,9,449


In [ ]:
score = model.evaluate(X_test, y_test)

print("\n   categorical_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 0s 5ms/step - loss: 0.3141 - accuracy: 0.9165

   categorical_crossentropy  ||   accuracy 
  ____________________________________
[0.3141055107116699, 0.9165253043174744]


   2. Defining the Architecture of 2-Layer of LSTM with more hyperparameter tunning

#### 2.1 First Model for 2-Layer of LSTM with more hyperparameter tunning

In [ ]:
# Initializing parameters
n_epochs = 30
n_batch = 16
n_classes = _count_classes(y_train)

# Bias regularizer value - we will use elasticnet
reg = L1L2(0.01, 0.01)

In [ ]:
# Model execution
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True,bias_regularizer=reg ))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 128, 48)           11136     
                                                                 
 batch_normalization_2 (Batc  (None, 128, 48)          192       
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 128, 48)           0         
                                                                 
 lstm_7 (LSTM)               (None, 32)                10368     
                                                                 
 dropout_7 (Dropout)         (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 6)                 198       
                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, y_train, batch_size=n_batch, validation_data=(X_test, y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 14s 17ms/step - loss: 1.5191 - accuracy: 0.6938 - val_loss: 0.9805 - val_accuracy: 0.8310
Epoch 2/30
460/460 [==============================] - 7s 16ms/step - loss: 0.7182 - accuracy: 0.8690 - val_loss: 0.5508 - val_accuracy: 0.8904
Epoch 3/30
460/460 [==============================] - 6s 13ms/step - loss: 0.3774 - accuracy: 0.9128 - val_loss: 0.3188 - val_accuracy: 0.8965
Epoch 4/30
460/460 [==============================] - 8s 17ms/step - loss: 0.2595 - accuracy: 0.9225 - val_loss: 0.4659 - val_accuracy: 0.8405
Epoch 5/30
460/460 [==============================] - 6s 14ms/step - loss: 0.2368 - accuracy: 0.9187 - val_loss: 0.4207 - val_accuracy: 0.8565
Epoch 6/30
460/460 [==============================] - 8s 17ms/step - loss: 0.2269 - accuracy: 0.9234 - val_loss: 0.2667 - val_accuracy: 0.9036
Epoch 7/30
460/460 [==============================] - 6s 14ms/step - loss: 0.1687 - accuracy: 0.9374 - val_loss: 0.2469 - val_accuracy: 0.912

In [ ]:
# Confusion Matrix
confusion_matrix(y_test, model.predict(X_test))

93/93 [==============================] - 2s 7ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,2,419,67,0,0,3
STANDING,0,128,404,0,0,0
WALKING,0,0,0,472,19,5
WALKING_DOWNSTAIRS,0,0,0,2,415,3
WALKING_UPSTAIRS,0,0,0,11,16,444


In [ ]:
score = model.evaluate(X_test, y_test)

print("\n   categorica_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 6ms/step - loss: 0.2640 - accuracy: 0.9131

   categorica_crossentropy  ||   accuracy 
  ____________________________________
[0.2640216052532196, 0.9131320118904114]


#### 2.2 Second Model for 2-Layer of LSTM with more hyperparameter tunning

In [ ]:
# Model execution
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 128, 64)           18944     
                                                                 
 batch_normalization_3 (Batc  (None, 128, 64)          256       
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_9 (LSTM)               (None, 48)                21696     
                                                                 
 dropout_9 (Dropout)         (None, 48)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                 294       
                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Training the model
model.fit(X_train, y_train, batch_size=n_batch, validation_data=(X_test, y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 13s 19ms/step - loss: 1.6734 - accuracy: 0.7078 - val_loss: 1.5870 - val_accuracy: 0.5945
Epoch 2/30
460/460 [==============================] - 6s 14ms/step - loss: 0.7557 - accuracy: 0.8898 - val_loss: 0.4793 - val_accuracy: 0.9179
Epoch 3/30
460/460 [==============================] - 8s 16ms/step - loss: 0.4011 - accuracy: 0.9135 - val_loss: 0.3697 - val_accuracy: 0.8761
Epoch 4/30
460/460 [==============================] - 6s 14ms/step - loss: 0.2216 - accuracy: 0.9287 - val_loss: 0.3722 - val_accuracy: 0.8829
Epoch 5/30
460/460 [==============================] - 8s 17ms/step - loss: 0.1946 - accuracy: 0.9334 - val_loss: 0.4488 - val_accuracy: 0.8446
Epoch 6/30
460/460 [==============================] - 6s 13ms/step - loss: 0.1719 - accuracy: 0.9381 - val_loss: 0.2355 - val_accuracy: 0.9128
Epoch 7/30
460/460 [==============================] - 7s 16ms/step - loss: 0.1631 - accuracy: 0.9399 - val_loss: 0.2094 - val_accuracy: 0.932

In [ ]:
# Confusion Matrix
confusion_matrix(y_test, model.predict(X_test))

93/93 [==============================] - 2s 7ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,0,407,82,0,0,2
STANDING,0,94,438,0,0,0
WALKING,0,0,0,447,46,3
WALKING_DOWNSTAIRS,0,0,0,1,419,0
WALKING_UPSTAIRS,0,2,0,1,27,441


In [ ]:
score = model.evaluate(X_test, y_test)

print("\n   categorical_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 9ms/step - loss: 0.3802 - accuracy: 0.9125

   categorical_crossentropy  ||   accuracy 
  ____________________________________
[0.38021206855773926, 0.9124533534049988]
